## KOBERT 감성분석
- 사전(Vocabulary)
- 크기 : 8,002
- 한글 위키 + 뉴스 텍스트 기반으로 학습한 토크나이저(SentencePiece)
- Less number of parameters(92M < 110M )

## 1. Library Import

In [142]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

##GPU 사용 시
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

## 2. Test Dataset Load

In [143]:
test_data = pd.read_csv('data/youtube_test_policy1.txt', sep='\t', header=None)
print(test_data.shape)
test_x, test_y = test_data[0], test_data[1]
print(test_x.shape, test_y.shape)

# num label to sentiment label
def num_to_sentiment_str(results):
    dic = {2:'positive', 1:'neutral',0:'negative'}
    convert_value = [dic[i] for i in results]
    return convert_value

pd.Series(num_to_sentiment_str(test_data[1])).value_counts()

(184, 2)
(184,) (184,)


negative    131
positive     35
neutral      18
dtype: int64

In [144]:
# Dataset Class 선언
class BERTDataset(Dataset):
    # dataset, 0, 1, tokenizer, max_len, True, False
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

## 3-1. 학습 전 기본 가중치 Get Model  & Tokenizer

In [145]:
# bert model
# vocab - Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = get_tokenizer() # str => /home/neuralworks/kobert/kobert_news_wiki_ko_cased-1087f8699e.spiece
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

print('=== Tokenizer 예제 ===')
tok('안녕하세요. 저는 김선민 입니다')

using cached model
using cached model
using cached model
=== Tokenizer 예제 ===


['▁안', '녕', '하세요', '▁', '.', '▁저', '는', '▁김', '선', '민', '▁', '입니다']

## 3-2. 학습 전 기본 가중치 Basic Model 선언

In [146]:
## model parameters Setting 
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# dataset parameters setting
max_len = 64 # 텍스트 데이터 최대 길이
batch_size = 4

In [147]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3, # 긍정 or 부정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
    
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

## 3-3. Predict

In [148]:
import pandas as pd
import numpy as np

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
    commnetslist = [] # 텍스트 데이터를 담을 리스트
    emo_list = [] # 감성 값을 담을 리스트
    for c in comment: # 모든 댓글
        commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
    pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
    pdData = pdData.values
    test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length 
        
        # 이때, out이 예측 결과 리스트
        out = model(token_ids, valid_length, segment_ids)
        # e는 n가지 실수 값으로 구성된 리스트
        # 0번 인덱스가 더 크면 부정, 긍정은 반대
        for e in out:
            value = int(torch.argmax(e).cpu())
            emo_list.append(value)
    return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

# Predict
youtube_data_list = test_data[0].tolist()
pred_y = getSentimentValue(youtube_data_list, tok, max_len, 8, device)
print('predict 3개 데이터 결과', results[:3], len(results))

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


## 3-4. Results

In [149]:
pred_y_basic = num_to_sentiment_str(pred_y)
test_y_basic = num_to_sentiment_str(test_y)

from sklearn.metrics import classification_report, accuracy_score
print(classification_report(pred_y_basic, test_y_basic))
print('Accuracy Score : ',accuracy_score(pred_y_basic, test_y_basic))

              precision    recall  f1-score   support

    negative       0.15      0.73      0.24        26
     neutral       0.28      0.08      0.12        63
    positive       0.60      0.22      0.32        95

    accuracy                           0.24       184
   macro avg       0.34      0.34      0.23       184
weighted avg       0.43      0.24      0.24       184

Accuracy Score :  0.24456521739130435


<hr>

## 4-1. Fine-tuning 학습 한 Modeling Loading

In [150]:
model = torch.load('model/kobert-policy-20.pt')
print('model loading completed')

model loading completed


## 4-2. Predict

In [151]:
import pandas as pd
import numpy as np

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
    commnetslist = [] # 텍스트 데이터를 담을 리스트
    emo_list = [] # 감성 값을 담을 리스트
    for c in comment: # 모든 댓글
        commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
    pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
    pdData = pdData.values
    test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length 
        
        # 이때, out이 예측 결과 리스트
        out = model(token_ids, valid_length, segment_ids)
        # e는 n가지 실수 값으로 구성된 리스트
        # 0번 인덱스가 더 크면 부정, 긍정은 반대
        for e in out:
            value = int(torch.argmax(e).cpu())
            emo_list.append(value)
    return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

# Predict
youtube_data_list = test_data[0].tolist()
pred_y = getSentimentValue(youtube_data_list, tok, max_len, 8, device)
print('predict 3개 데이터 결과', results[:3], len(results))

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


## 4.3 Results

In [152]:
pred_y_ft = num_to_sentiment_str(pred_y)
test_y_ft = num_to_sentiment_str(test_y)

from sklearn.metrics import classification_report, accuracy_score
print(classification_report(pred_y_ft, test_y_ft))
print('Accuracy Score : ', accuracy_score(pred_y_ft, test_y_ft))

              precision    recall  f1-score   support

    negative       0.94      0.83      0.88       149
     neutral       0.17      0.43      0.24         7
    positive       0.43      0.54      0.48        28

    accuracy                           0.77       184
   macro avg       0.51      0.60      0.53       184
weighted avg       0.83      0.77      0.79       184

Accuracy Score :  0.7663043478260869


## 5. 최종 정리

- 실험 결과
    - 현재 모델의 성능 측정을 위해 데이터셋을 7대 3으로 나누어 테스트 한 결과
    - Fine-tuning 시 기존 학습전보다 성능이 향상함을 보임.
    - Generalized 학습 된 네이버와, Fine-tuning 한 KoBERT가 어느정도 비슷한 성능을 보임.
    - 단 여기서 KoBERT는 아래와 같은 문제가 아직 남아있어 개선 가능의 여지가 많음.

- 추가 개선 가능 사항 및 필요 사항.
    - 데이터의 수가 비교적 적음 (일반적으로 자연어 Task가 일반화 되기 위해선 많은 데이터 학습이 필요) 
    - 긍정, 부정, 중립간의 라벨이 불균형해 비율이 낮은 라벨은 잘 분류를 못할 수 있음 => Balanced 한 데이터 수집 필요
    
- 추가
    - 한 도메인에(정치 or 식품) 여러 기사 댓글 등 정답이 분류된 라벨별 300개 이상 데이터로 실험 필요)

In [153]:
youtube_data = pd.read_excel('data/Youtube댓글정리_httpsyoutu.belm7p--F7eF0_이재명_모든경기도민에_재난지원금_정리완료.xlsx')
youtube_data = youtube_data.iloc[:-4]
youtube_data = youtube_data[['댓글','naver_결과', 'azure_결과', '결과', 'naver 비교', 'azure  비교']] # kobert_결과, 'kobert 비교'

size = int(len(youtube_data)*0.7)
youtube_data = youtube_data.iloc[size:]
print(youtube_data.shape)
youtube_data

(184, 6)


,댓글,naver_결과,azure_결과,결과,naver 비교,azure 비교
428,경기도에 사는것들이 없나보구나,negative,neutral,negative,T,F
429,민주당 대선후보들 이재명지사님빼고 다 이상함 다른시.도 민들이 보편지급하는걸 좋아...,negative,mixed,positive,F,F
430,저게 맞지,neutral,positive,positive,F,T
431,경기도 잘한다. ㅈㄹ 말고 반성해라,negative,positive,negative,T,F
432,지자체 공산당이네..,negative,neutral,negative,T,F
...,...,...,...,...,...,...
607,댓글들이 이재명지지자들이 많네요,neutral,neutral,positive,F,F
608,"재명아, 니 돈으로 주나 이런자는 대통 되서는 안됩니다 공짜 아닙니다 골병듭...",negative,negative,negative,T,T
609,이재명은 마음대로 할수 있는 경기도지사 절대 사표 내지 마라,negative,negative,negative,T,T
610,대통령 출마한사람이 할래면 다하지 쪽팔리다 경기도민은 야찍어 나머지는 윤 찍자 무흣,negative,neutral,negative,T,F


In [154]:
# 학습 전 
youtube_data['kobert_학습전 결과'] = pred_y_basic
youtube_data['kobert 학습전 비교'] = youtube_data['kobert_학습전 결과'] == youtube_data['결과']
youtube_data['kobert 학습전 비교'] = youtube_data['kobert 학습전 비교'].apply(lambda x : 'T' if x==True else 'F')

# 학습 후
youtube_data['kobert_학습후 결과'] = pred_y_ft
youtube_data['kobert 학습후 비교'] = youtube_data['kobert_학습후 결과'] == youtube_data['결과']
youtube_data['kobert 학습후 비교'] = youtube_data['kobert 학습후 비교'].apply(lambda x : 'T' if x==True else 'F')
youtube_data

,댓글,naver_결과,azure_결과,결과,naver 비교,azure 비교,kobert_학습전 결과,kobert 학습전 비교,kobert_학습후 결과,kobert 학습후 비교
428,경기도에 사는것들이 없나보구나,negative,neutral,negative,T,F,neutral,F,negative,T
429,민주당 대선후보들 이재명지사님빼고 다 이상함 다른시.도 민들이 보편지급하는걸 좋아...,negative,mixed,positive,F,F,positive,T,negative,F
430,저게 맞지,neutral,positive,positive,F,T,negative,F,positive,T
431,경기도 잘한다. ㅈㄹ 말고 반성해라,negative,positive,negative,T,F,neutral,F,negative,T
432,지자체 공산당이네..,negative,neutral,negative,T,F,neutral,F,negative,T
...,...,...,...,...,...,...,...,...,...,...
607,댓글들이 이재명지지자들이 많네요,neutral,neutral,positive,F,F,negative,F,negative,F
608,"재명아, 니 돈으로 주나 이런자는 대통 되서는 안됩니다 공짜 아닙니다 골병듭...",negative,negative,negative,T,T,positive,F,negative,T
609,이재명은 마음대로 할수 있는 경기도지사 절대 사표 내지 마라,negative,negative,negative,T,T,neutral,F,negative,T
610,대통령 출마한사람이 할래면 다하지 쪽팔리다 경기도민은 야찍어 나머지는 윤 찍자 무흣,negative,neutral,negative,T,F,positive,F,negative,T


## 6. 최종 정리 Score 및 csv 만들기

In [165]:
az_score = np.round(100*len(youtube_data[youtube_data['azure  비교']=='T']) / len(youtube_data),2)
nv_score = np.round(100*len(youtube_data[youtube_data['naver 비교']=='T']) / len(youtube_data),2)
kb_score1 = np.round(100*len(youtube_data[youtube_data['kobert 학습전 비교']=='T']) / len(youtube_data),2)
kb_score2 = np.round(100*len(youtube_data[youtube_data['kobert 학습후 비교']=='T']) / len(youtube_data),2)
print('Azure API Score:', az_score, '\nNaver API Score:', nv_score, '\nKobert 학습전 Score:', kb_score1, '\nKobert 학습후 Score:', kb_score2)

Azure API Score: 50.54 
Naver API Score: 77.17 
Kobert 학습전 Score: 24.46 
Kobert 학습후 Score: 76.63


In [166]:
youtube_data.to_csv('youtube_정치_이재명..모델_학습_전후_결과_비교.csv', encoding='utf-8-sig')